In [ ]:
# 📦 Импорт библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col, to_timestamp, date_format
import matplotlib.pyplot as plt

# 🚀 Запуск Spark
spark = SparkSession.builder \
    .appName("WebLogAnalysis") \
    .getOrCreate()

# 📂 Загрузка логов
log_file = "data/sample_logs.log"
logs_df = spark.read.text(log_file)

# 🧹 Парсинг логов с помощью регулярного выражения
log_pattern = r'^(\S+) - - \[([^\]]+)\] "(\S+)\s(\S+)\s(\S+)" (\d{3}) (\S+)'

parsed_df = logs_df.select(
    regexp_extract('value', log_pattern, 1).alias('ip'),
    regexp_extract('value', log_pattern, 2).alias('timestamp'),
    regexp_extract('value', log_pattern, 3).alias('method'),
    regexp_extract('value', log_pattern, 4).alias('url'),
    regexp_extract('value', log_pattern, 5).alias('protocol'),
    regexp_extract('value', log_pattern, 6).cast("integer").alias('status'),
    regexp_extract('value', log_pattern, 7).alias('size')
)

# 🛠 Обработка размера (преобразуем "-" в 0)
parsed_df = parsed_df.withColumn("size", col("size").cast("integer").na.fill(0))

# 👀 Просмотр нескольких строк
parsed_df.show(5, truncate=False)

In [ ]:
# 📊 Топ-10 IP по количеству запросов
top_ips = parsed_df.groupBy("ip").count().orderBy("count", ascending=False)
top_ips.show(10)

In [ ]:
# 📊 Распределение по HTTP-кодам
status_counts = parsed_df.groupBy("status").count().orderBy("count", ascending=False)
status_counts.show()

In [ ]:
# ⏰ Анализ активности по часам
df_with_time = parsed_df.withColumn(
    "datetime", to_timestamp("timestamp", "dd/MMM/yyyy:HH:mm:ss")
)

hourly = df_with_time.groupBy(
    date_format("datetime", "yyyy-MM-dd HH:00").alias("hour")
).count().orderBy("hour")

hourly.show(truncate=False)

In [ ]:
# 📈 Визуализация: HTTP-коды
status_pd = status_counts.toPandas()

plt.figure(figsize=(8, 5))
plt.bar(status_pd["status"].astype(str), status_pd["count"], color="skyblue")
plt.xlabel("HTTP Status Code")
plt.ylabel("Count")
plt.title("Распределение HTTP-кодов")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 📈 Визуализация: запросы по часам
hourly_pd = hourly.toPandas()

plt.figure(figsize=(10, 5))
plt.plot(hourly_pd["hour"], hourly_pd["count"], marker='o')
plt.xticks(rotation=45)
plt.xlabel("Hour")
plt.ylabel("Request Count")
plt.title("Активность по часам")
plt.grid(True)
plt.tight_layout()
plt.show()